In [1]:
import pandas as pd
from data_processing import save_data

In [2]:
url = "https://www.ofgem.gov.uk/publications/renewables-obligation-ro-buy-out-price-mutualisation-threshold-and-mutualisation-ceilings-2024-2025"

In [3]:
all_tables = pd.read_html(url)

In [4]:
# Check the length of tables list
print(len(all_tables))

2


drop northern ireland data from the table.

In [5]:
# Check the first table
# The buy-out prices and obligation levels for recent years are shown in the table below:
ROC = all_tables[0]
ROC


,Obligation Period (1 April - 31 March),Buy-out Price,Obligation For England and Wales and Scotland (ROCs/MWh),Obligation For Northern Ireland (ROCs/MWh)
0,2010-2011,£36.99,0.111,0.0427
1,2011-2012,£38.69,0.124,0.0550
2,2012-2013,£40.71,0.158,0.0810
3,2013-2014,£42.02,0.206,0.0970
4,2014-2015,£43.30,0.244,0.1070
5,2015-2016,£44.33,0.290,0.1190
6,2016-2017,£44.77,0.348,0.1420
7,2017-2018,£45.58,0.409,0.1670
8,2018-2019,£47.22,0.468,0.1850
9,2019-2020,£48.78,0.484,0.1900


In [6]:
ROC_processed = ROC.drop('Obligation For Northern Ireland (ROCs/MWh)', axis=1)
# Extract the first year from the 'YearRange' column
ROC_processed['year'] = ROC_processed['Obligation Period (1 April - 31 March)'].str.split('-').str[0]

# Create a new date column with April 1st of each year
ROC_processed['date'] = pd.to_datetime(ROC_processed['year'] + '-04-01')

# Remove currency symbol and convert to numeric
ROC_processed['Buy-out Price'] = pd.to_numeric(ROC_processed['Buy-out Price'].str.replace('£', ''))

# Drop the original 'YearRange' column if needed
ROC_processed = ROC_processed.drop(['Obligation Period (1 April - 31 March)', 'year'], axis=1)

ROC_processed = ROC_processed.rename(columns={'Buy-out Price':'ROC Buy-out Price (£)'})

# Replace non-breaking spaces with regular spaces
ROC_processed.columns = ROC_processed.columns.str.replace('\xa0', ' ', regex=True)

# Now extract the numeric part as intended
ROC_processed['Obligation For England and Wales and Scotland (ROCs/MWh)'] = (
    ROC_processed['Obligation For England and Wales and Scotland (ROCs/MWh)']
    .str.extract(r'([\d\.]+)')
    .astype(float)
)

ROC_processed['country_cd'] = 'UK'
ROC_processed

,ROC Buy-out Price (£),Obligation For England and Wales and Scotland (ROCs/MWh),date,country_cd
0,36.99,0.111,2010-04-01,UK
1,38.69,0.124,2011-04-01,UK
2,40.71,0.158,2012-04-01,UK
3,42.02,0.206,2013-04-01,UK
4,43.30,0.244,2014-04-01,UK
5,44.33,0.290,2015-04-01,UK
6,44.77,0.348,2016-04-01,UK
7,45.58,0.409,2017-04-01,UK
8,47.22,0.468,2018-04-01,UK
9,48.78,0.484,2019-04-01,UK


In [7]:
# save as csv
save_data({'roc':ROC}, data_type='raw')
save_data({'roc':ROC_processed})